In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import datashader as ds
from hvplot import pandas
from holoviews import opts #, dim 
from holoviews.element import tiles
from datetime import datetime, timedelta

In [ ]:
opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=700, frame_height=300))

BG_TILES = tiles.CartoLight()
EVERID = 235

In [ ]:
df = pd.read_csv('data/boat-positions.csv', parse_dates=True)
df['t'] = pd.to_datetime(df['t'], format='%d/%m/%Y %H:%M')
df

In [ ]:
tmp = df.copy()
tmp.loc[:, 'x'], tmp.loc[:, 'y'] = ds.utils.lnglat_to_meters(tmp.lon, tmp.lat)
BG_TILES * tmp.hvplot.scatter(title='Density map', x='x', y='y', datashade=True)

In [ ]:
tc = mpd.TrajectoryCollection(df, 'id', t='t', x='lon', y='lat', crs=4326)
tc

In [ ]:
tc.to_traj_gdf().hvplot(title='All trajectories', geo=True, tiles='OSM')

In [ ]:
tc.get_trajectory(1)#.hvplot(geo=True)

In [ ]:
tc.add_speed(units=('km','h'))

In [ ]:
df.id.max()

In [ ]:
traj_id = 15
tc.get_trajectory(traj_id).hvplot(title=f'Trajectory {traj_id}', c='speed', cmap='Viridis', 
                                  clim=(0,10), tiles='CartoLight') 

In [ ]:
traj_id = EVERID  
( tc.to_traj_gdf().hvplot(title=f'Trajectory {traj_id} on top of all other trajectories', 
                          geo=True, tiles='OSM', color='white', alpha=0.5) *
tc.get_trajectory(traj_id).hvplot(c='speed', cmap='Viridis', clim=(0,7), tiles=None) )

In [ ]:
( BG_TILES * 
  tmp.hvplot.scatter(title=f'Trajectory {traj_id} on top of density map',
                    x='x', y='y', datashade=True, cmap=['lightpink','hotpink','darkblue']) *
  tc.get_trajectory(traj_id).hvplot(c='speed', cmap='Viridis', clim=(0,7), tiles=None) )

In [ ]:
tc.to_point_gdf().hvplot.scatter(x='t', y='speed')

In [ ]:
stop_pts = mpd.TrajectoryStopDetector(tc).get_stop_points(min_duration=timedelta(hours=3), max_diameter=1000)
stop_pts['duration_h'] = stop_pts['duration_s'] / 3600
stop_pts

In [ ]:
( pd.DataFrame(stop_pts).hvplot.scatter(
    title='Stop start & duration (in hours)', x='start_time', y='duration_h') *
  pd.DataFrame(stop_pts[stop_pts['traj_id']==EVERID]).hvplot.scatter(
      x='start_time', y='duration_h', color='red', size=200) )

In [ ]:
stop_pts[stop_pts.start_time > datetime(2021,3,23,5,0,0)]\
    .sort_values('duration_s', ascending=False)\
    .head(7)\
    .style.background_gradient(cmap='Reds')